<a href="https://colab.research.google.com/github/prevelat/Machine_Learning/blob/master/tf_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import os
import json
from pandas.io.json import json_normalize
import sys
# import matplotlib.pyplot as plt
# from IPython.display import clear_output
# from six.moves import urllib
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from keras import layers
%load_ext tensorboard

TensorFlow 2.x selected.


Using TensorFlow backend.


In [0]:
#-------------------- get data --------------------

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#-------------------- get data --------------------

os.chdir("drive/My Drive/Colab Notebooks")
!ls

 100k_quantum_circuits_valid_json.json
 boostedTrees.ipynb
'Copy of firstTest.ipynb'
'Copy of quantum tf.estimator 0 - predict stdev.ipynb'
'Copy of validation.ipynb'
 Ex_Validation.ipynb
 firstTry.ipynb
 Ivan_qc_multilabelhead.ipynb
 Keras
 linearModel.ipynb
 premadeEstimator.ipynb
 random_5_qubit_circuits_100k_quantum_circuits_valid_json.json.zip
 RMSProp_softmax
 RMSProp_softmax.ipynb
 simpleModels.ipynb
 testKeras


In [0]:
#-------------------- get data --------------------

data = pd.read_json('100k_quantum_circuits_valid_json.json')
# data

In [0]:
#-------------------- preprocess data --------------------

def prepare_gate_dataframe(gate_dataframe, gate_suffix):

  """
  Explain the function
  Arguments:
      Explain arguments
  Returns:
      Explain returns
  """

    #--------------- for the gate data ------------------
    #------------ split all numerical data --------------
    #--------------- into a new key:value ---------------
    #------------- pair for the new (dict) --------------
    #--------------- dataframe created ------------------

  normalized_gate_dataframe = json_normalize(gate_dataframe)
  dataframe = pd.DataFrame()

  for key in normalized_gate_dataframe:
    if key == 'Gate_Number' or key == 'Gate_Type':
      continue
    newkey = key + gate_suffix
    dataframe[newkey] = normalized_gate_dataframe[key]

  return dataframe

def normalize_dataframe(dataframe):

  """
  Explain the function
  Arguments:
      Explain arguments
  Returns:
      Explain returns
  """

    #------------ split labels (statevector) ------------
    #------------- and features (gate data) -------------
    #--------------- preprocessing labels ---------------
    #------------ for probability predicting ------------
    #------------- creating a new dataframe -------------

  normalized_data = json_normalize(dataframe['statevector'])
  for key in normalized_data:
    normalized_data[key] /= 1024
  for key in dataframe:
    if 'gate' in key:
      suffix = '_%s' % key[-2:]
      gate_dataframe = prepare_gate_dataframe(dataframe[key], suffix)
      normalized_data = normalized_data.join(gate_dataframe)
  return normalized_data

normalized_dataframe = normalize_dataframe(data)

In [0]:
#-------------------- check --------------------

normalized_dataframe.head()

,state_11111,state_10111,state_11011,state_11100,state_01001,state_01100,state_00000,state_00100,state_01000,state_11110,state_11000,state_00001,state_00111,state_00010,state_11010,state_10000,state_10110,state_10010,state_00101,state_10100,state_01111,state_11001,state_10001,state_00011,state_10101,state_01010,state_11101,state_10011,state_01110,state_01101,state_00110,state_01011,Angle_3_11,Control_11,Angle_1_11,Target_11,Angle_2_11,Angle_3_30,Control_30,Angle_2_30,...,Angle_3_14,Control_14,Angle_1_14,Target_14,Angle_2_14,Angle_3_26,Control_26,Angle_1_26,Target_26,Angle_2_26,Angle_3_07,Control_07,Angle_1_07,Target_07,Angle_2_07,Angle_3_03,Control_03,Angle_1_03,Target_03,Angle_2_03,Angle_3_21,Control_21,Angle_1_21,Target_21,Angle_2_21,Angle_3_22,Control_22,Angle_1_22,Target_22,Angle_2_22,Angle_3_36,Control_36,Angle_2_36,Target_36,Angle_1_36,Angle_3_04,Control_04,Angle_1_04,Target_04,Angle_2_04
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-1.000000,0,-1.0,1,-1.0,-1.000000,-1,-1.0,...,0.709553,3,0.0,-1,0.0,-1.0,1,-1.000000,-1,-1.0,-1.0000,1,-1.0,0,-1.0,0.713632,2,0.0,-1,0.0,-0.709553,2,-0.804929,-1,0.0,-1.000000,3,-1.000000,2,-1.0,-1,-1,-1,-1,-1,1.00000,3,0.5,-1,0.0
1,0.0,0.0,0.0,0.000977,0.000000,0.006836,0.0,0.005859,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.269531,0.0,0.0,0.0,0.0,0.000000,0.068359,0.0,0.123047,0.0,0.0,0.525391,0.000000,0.000000,1.000000,2,1.0,-1,0.0,0.500000,4,-0.5,...,1.088594,1,0.0,-1,0.0,0.5,3,1.392736,-1,-0.5,-1.0000,3,-1.0,2,-1.0,0.779094,0,0.0,-1,0.0,-1.000000,1,-1.000000,0,-1.0,-1.109076,0,-0.735356,-1,0.0,-1,-1,-1,-1,-1,0.30387,3,0.0,-1,0.0
2,0.0,0.0,0.0,0.000000,0.461914,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.495117,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.042969,-0.584628,2,0.0,-1,0.0,0.500000,4,0.0,...,1.000000,3,1.0,-1,0.0,-0.5,2,0.000000,-1,0.0,0.8808,1,0.0,-1,0.0,-1.000000,0,-1.0,1,-1.0,1.244226,4,0.000000,-1,0.0,-1.000000,4,-1.000000,2,-1.0,-1,4,-1,-1,-1,-0.88080,1,0.0,-1,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000977,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.999023,0.000000,0.102862,4,0.0,-1,0.0,-1.000000,-1,-1.0,...,-1.000000,2,-1.0,4,-1.0,-1.0,4,-1.000000,-1,-1.0,1.0000,4,0.5,-1,0.0,0.368304,2,0.0,-1,0.0,-1.000000,0,-1.000000,4,-1.0,-1.000000,0,-1.000000,-1,-1.0,-1,-1,-1,-1,-1,-1.00000,3,-1.0,2,-1.0
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-1.000000,2,-1.0,4,-1.0,1.950577,4,0.0,...,1.000000,4,0.5,-1,0.0,-1.0,0,-1.000000,4,-1.0,1.0000,3,0.5,-1,0.0,0.984549,2,0.0,-1,0.0,-0.500000,4,0.000000,-1,0.0,0.531491,4,0.000000,-1,0.0,-1,4,-1,-1,-1,-1.00000,3,-1.0,1,-1.0


In [0]:
#-------------------- split features and labels --------------------
def split_data(data):

  """
  Explain the function
  Arguments:
      Explain arguments
  Returns:
      Explain returns
  """

  features = data.filter(regex='^(?!state)', axis=1)
  labels = data.filter(regex='^state', axis=1)
  return (features, labels)

#-------------------- get desired amount of train data --------------------

train_data = normalized_dataframe[0:70000]
# train_data

#-------------------- get desired amount of validation data --------------------

validation_data = normalized_dataframe[70000:90000]
# validation_data

#-------------------- get desired amount of testing data --------------------

test_data = normalized_dataframe[90000:100000]
# test_data

#-------------------- split features and targets from data --------------------

training_features, training_targets = split_data(train_data)
validation_features, validation_targets = split_data(validation_data)
test_features, test_targets = split_data(test_data)

In [0]:
#-------------------- check --------------------

training_features
# training_targets.values
# validation_features
# validation_targets
# test_features
# test_targets

,Angle_3_11,Control_11,Angle_1_11,Target_11,Angle_2_11,Angle_3_30,Control_30,Angle_2_30,Target_30,Angle_1_30,Angle_3_28,Control_28,Angle_1_28,Target_28,Angle_2_28,Angle_3_08,Control_08,Angle_1_08,Target_08,Angle_2_08,Angle_3_12,Control_12,Angle_1_12,Target_12,Angle_2_12,Angle_3_20,Control_20,Angle_1_20,Target_20,Angle_2_20,Angle_3_15,Control_15,Angle_1_15,Target_15,Angle_2_15,Angle_3_32,Control_32,Angle_2_32,Target_32,Angle_1_32,...,Angle_3_14,Control_14,Angle_1_14,Target_14,Angle_2_14,Angle_3_26,Control_26,Angle_1_26,Target_26,Angle_2_26,Angle_3_07,Control_07,Angle_1_07,Target_07,Angle_2_07,Angle_3_03,Control_03,Angle_1_03,Target_03,Angle_2_03,Angle_3_21,Control_21,Angle_1_21,Target_21,Angle_2_21,Angle_3_22,Control_22,Angle_1_22,Target_22,Angle_2_22,Angle_3_36,Control_36,Angle_2_36,Target_36,Angle_1_36,Angle_3_04,Control_04,Angle_1_04,Target_04,Angle_2_04
0,-1.000000,0,-1.0,1,-1.0,-1.000000,-1,-1.0,-1,-1.000000,-1.000000,3,-1.000000,-1,-1.0000,-1.000000,0,-1.0,1,-1.0,1.000000,3,0.500000,-1,0.00000,-1.000000,3,-1.000000,2,-1.0,0.500000,4,1.0,-1,0.5,-1.0,-1,-1.0,-1,-1.0,...,0.709553,3,0.0,-1,0.0,-1.000000,1,-1.000000,-1,-1.0,-1.000000,1,-1.000000,0,-1.0,0.713632,2,0.0,-1,0.0,-0.709553,2,-0.804929,-1,0.0,-1.000000,3,-1.000000,2,-1.000000,-1,-1,-1,-1,-1,1.000000,3,0.500000,-1,0.000000
1,1.000000,2,1.0,-1,0.0,0.500000,4,-0.5,-1,0.712385,0.500000,4,1.000000,-1,0.5000,0.303870,2,0.0,-1,0.0,1.000000,3,0.500000,-1,0.00000,1.109076,1,0.000000,-1,0.0,1.000000,3,0.5,-1,0.0,-1.0,1,-1.0,-1,-1.0,...,1.088594,1,0.0,-1,0.0,0.500000,3,1.392736,-1,-0.5,-1.000000,3,-1.000000,2,-1.0,0.779094,0,0.0,-1,0.0,-1.000000,1,-1.000000,0,-1.0,-1.109076,0,-0.735356,-1,0.000000,-1,-1,-1,-1,-1,0.303870,3,0.000000,-1,0.000000
2,-0.584628,2,0.0,-1,0.0,0.500000,4,0.0,-1,0.000000,-1.000000,0,-1.000000,2,-1.0000,1.851623,1,0.0,-1,0.0,0.770050,3,0.000000,-1,0.00000,-0.500000,4,0.000000,-1,0.0,1.779629,3,0.0,-1,0.0,-1.0,0,-1.0,-1,-1.0,...,1.000000,3,1.0,-1,0.0,-0.500000,2,0.000000,-1,0.0,0.880800,1,0.000000,-1,0.0,-1.000000,0,-1.0,1,-1.0,1.244226,4,0.000000,-1,0.0,-1.000000,4,-1.000000,2,-1.000000,-1,4,-1,-1,-1,-0.880800,1,0.000000,-1,0.000000
3,0.102862,4,0.0,-1,0.0,-1.000000,-1,-1.0,-1,-1.000000,-1.000000,-1,-1.000000,-1,-1.0000,-1.000000,1,-1.0,4,-1.0,-1.000000,2,-1.000000,4,-1.00000,-0.598907,4,0.000000,-1,0.0,-0.500000,2,0.0,-1,0.0,-1.0,-1,-1.0,-1,-1.0,...,-1.000000,2,-1.0,4,-1.0,-1.000000,4,-1.000000,-1,-1.0,1.000000,4,0.500000,-1,0.0,0.368304,2,0.0,-1,0.0,-1.000000,0,-1.000000,4,-1.0,-1.000000,0,-1.000000,-1,-1.000000,-1,-1,-1,-1,-1,-1.000000,3,-1.000000,2,-1.000000
4,-1.000000,2,-1.0,4,-1.0,1.950577,4,0.0,-1,0.000000,-1.000000,2,-1.000000,1,-1.0000,-0.347541,4,0.0,-1,0.0,0.000000,4,0.711135,-1,1.33209,1.000000,4,0.500000,-1,0.0,0.250000,4,0.0,-1,0.0,-1.0,0,-1.0,-1,-1.0,...,1.000000,4,0.5,-1,0.0,-1.000000,0,-1.000000,4,-1.0,1.000000,3,0.500000,-1,0.0,0.984549,2,0.0,-1,0.0,-0.500000,4,0.000000,-1,0.0,0.531491,4,0.000000,-1,0.000000,-1,4,-1,-1,-1,-1.000000,3,-1.000000,1,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1.267215,2,0.0,-1,0.0,-1.000000,4,-1.0,-1,-1.000000,-1.000000,2,-1.000000,-1,-1.0000,-1.000000,1,-1.0,2,-1.0,-1.000000,0,-1.000000,2,-1.00000,0.375947,3,0.000000,-1,0.0,1.000000,3,0.0,-1,0.0,-1.0,-1,-1.0,-1,-1.0,...,0.625458,2,0.0,-1,0.0,-1.000000,0,-1.000000,-1,-1.0,0.234138,2,0.000000,-1,0.0,-1.000000,1,-1.0,0,-1.0,-1.000000,4,-1.000000,3,-1.0,-0.500000,3,0.000000,-1,0.000000,-1,-1,-1,-1,-1,0.908294,0,0.000000,-1,0.000000
59996,-1.000000,4,-1.0,3,-1.0,-1.000000,-1,-1.0,-1,-1.000000,-1.000000,-1,-1.000000,-1,-1.0000,0.057549,4,0.0,-1,0.0,0.057549,3,0.000000,-1,0.00000,-1.000000,2,-1.000000,-1,-1.0,-0.111185,3,0.0,-1,0.0,-1.0,-1,-1.0,-1,-1.0,...,-1.000000,2,

In [0]:
#-------------------- Run TensorBoard --------------------

model_dir = 'Keras'
!rm -rf $model_dir
tf.keras.backend.clear_session()

In [0]:
%tensorboard --logdir $model_dir

In [0]:
!kill 16535

/bin/bash: line 0: kill: (16535) - No such process


In [0]:
#-------------------- Build model using tf.keras API --------------------

inputs = tf.keras.Input(shape=(205,))
x = tf.keras.layers.Dense(128, activation='sigmoid')(inputs)
x = tf.keras.layers.Dense(56, activation='sigmoid')(x)
outputs = tf.keras.layers.Dense(32, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop())

In [0]:
#-------------------- Visualize Model --------------------

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 205)]             0         
_________________________________________________________________
dense (Dense)                (None, 41)                8446      
_________________________________________________________________
dense_1 (Dense)              (None, 41)                1722      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1344      
Total params: 11,512
Trainable params: 11,512
Non-trainable params: 0
_________________________________________________________________


In [0]:
#-------------------- Train and Evaluate --------------------

model.fit(
    x=training_features.values,
    y=training_targets.values,
    epochs=1000,
    verbose=2,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir=model_dir)],
    validation_data=(validation_features.values, validation_targets.values))

Train on 60000 samples, validate on 30000 samples
Epoch 1/120
60000/60000 - 10s - loss: 2.6241 - val_loss: 2.5183
Epoch 2/120
60000/60000 - 8s - loss: 2.4889 - val_loss: 2.4460
Epoch 3/120
60000/60000 - 8s - loss: 2.4216 - val_loss: 2.3895
Epoch 4/120
60000/60000 - 8s - loss: 2.3758 - val_loss: 2.3541
Epoch 5/120
60000/60000 - 8s - loss: 2.3428 - val_loss: 2.3319
Epoch 6/120
60000/60000 - 8s - loss: 2.3167 - val_loss: 2.3283
Epoch 7/120
60000/60000 - 8s - loss: 2.2958 - val_loss: 2.2949
Epoch 8/120
60000/60000 - 8s - loss: 2.2781 - val_loss: 2.3111
Epoch 9/120
60000/60000 - 8s - loss: 2.2632 - val_loss: 2.2756
Epoch 10/120
60000/60000 - 8s - loss: 2.2520 - val_loss: 2.2574
Epoch 11/120
60000/60000 - 8s - loss: 2.2399 - val_loss: 2.2739
Epoch 12/120
60000/60000 - 8s - loss: 2.2302 - val_loss: 2.2484
Epoch 13/120
60000/60000 - 8s - loss: 2.2213 - val_loss: 2.2387
Epoch 14/120
60000/60000 - 8s - loss: 2.2141 - val_loss: 2.2221
Epoch 15/120
60000/60000 - 8s - loss: 2.2061 - val_loss: 2.224

In [0]:
#-------------------- Predict --------------------

result = model.predict(test_features.values, batch_size=32)

In [0]:
print(result[0])
print(test_targets.values[0])

[0.01082209 0.01473893 0.01423498 0.01815314 0.02821681 0.03733431
 0.10076497 0.06758076 0.05366572 0.01391802 0.02625365 0.05075854
 0.02113301 0.0687312  0.0193832  0.05051554 0.02235908 0.0354587
 0.03430501 0.0351121  0.01446014 0.01720646 0.02622852 0.03379082
 0.01940036 0.03140101 0.01286656 0.01907971 0.02248499 0.01993557
 0.03936526 0.02034079]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.46191406 0.
 0.53808594 0.        ]
